In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from skimage.feature import hog

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import joblib


In [2]:
import os
import cv2
import numpy as np
from skimage.feature import hog


DATASET_PATH = r"C:\Users\HP\Desktop\Signature Authenticator\sign_data\train"

IMG_SIZE = (200, 100)

X = []
Y = []


def extract_features(image):

    image = cv2.resize(image, IMG_SIZE)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    features = hog(gray,
                   orientations=9,
                   pixels_per_cell=(8,8),
                   cells_per_block=(2,2))
    
    return features


folders = os.listdir(DATASET_PATH)

print("Total Folders:", len(folders))


for folder in folders:
    
    folder_path = os.path.join(DATASET_PATH, folder)
    
    if not os.path.isdir(folder_path):
        continue


    # Label
    if "_forg" in folder.lower():
        label = 0   # Forged
    else:
        label = 1   # Genuine


    for file in os.listdir(folder_path):
        
        img_path = os.path.join(folder_path, file)
        
        img = cv2.imread(img_path)
        
        if img is None:
            continue
        
        features = extract_features(img)
        
        X.append(features)
        Y.append(label)


X = np.array(X)
Y = np.array(Y)

print("Total Images:", len(X))
print("Genuine:", sum(Y==1))
print("Forged:", sum(Y==0))
print("Feature Size:", X.shape[1])


Total Folders: 128
Total Images: 1649
Genuine: 887
Forged: 762
Feature Size: 9504


In [3]:
# -------------------------------
# Split Data
# -------------------------------

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, Y,
    test_size=0.2,
    random_state=42,
    stratify=Y
)

print("Train:", len(X_train))
print("Test :", len(X_test))


# -------------------------------
# Train SVM
# -------------------------------

from sklearn.svm import SVC

model = SVC(
    kernel="rbf",
    C=10,
    gamma="scale",
    probability=True
)

print("Training started...")

model.fit(X_train, y_train)

print("Training finished ✅")


# -------------------------------
# Evaluate
# -------------------------------

from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)

print("\nAccuracy:", acc*100, "%\n")

print("Report:\n")
print(classification_report(y_test, y_pred))


# -------------------------------
# Save Model
# -------------------------------

import joblib

joblib.dump(model, "signature_model.pkl")

print("\nModel saved as signature_model.pkl ✅")


Train: 1319
Test : 330
Training started...
Training finished ✅

Accuracy: 88.7878787878788 %

Report:

              precision    recall  f1-score   support

           0       0.90      0.86      0.88       152
           1       0.88      0.92      0.90       178

    accuracy                           0.89       330
   macro avg       0.89      0.89      0.89       330
weighted avg       0.89      0.89      0.89       330


Model saved as signature_model.pkl ✅
